In [ ]:
import codecs
import os
import sys
import threading
import serial
from serial.tools.list_ports import comports

1. Gimbal control

In [ ]:
from a2gmeasurements import GimbalRS2
import time, timeit
import numpy as np

myGimbal = GimbalRS2()
myGimbal.start_thread_gimbal()

Finding required time for gimbal to answer a position request

In [ ]:
from a2gmeasurements import GimbalRS2
import time, timeit
import numpy as np

myGimbal = GimbalRS2()
myGimbal.start_thread_gimbal()

assumed_time_for_execute_time = 0.0015 # s
gimbal_speed = (1800/3.47) # DEG/s
condition_continue = True
tol = gimbal_speed * assumed_time_for_execute_time
cnt = 0
myGimbal.request_current_position()
time.sleep(0.0015)
last_yaw = myGimbal.yaw 
last_roll = myGimbal.roll
print('Outside: ',last_yaw, last_roll)
myGimbal.setPosControl(yaw=1250, roll=0, pitch=0)
time.sleep(1)
while(condition_continue):
    print('Execution time: ', timeit.default_timer() - start)
    myGimbal.request_current_position()
    time.sleep(0.0015)
    print('Inside: ', myGimbal.yaw, myGimbal.roll)
    
    if np.abs(last_yaw - myGimbal.yaw) < tol and np.abs(last_roll - myGimbal.roll) < tol:
        condition_continue = False
    else:
        last_yaw = myGimbal.yaw
        last_roll = myGimbal.roll
        start = timeit.default_timer()
    cnt = cnt+1
    print(cnt)
    


In [ ]:
#myGimbal.request_current_position()
myGimbal.setPosControl(yaw=0, roll=0, pitch=0)

In [ ]:
myGimbal.stop_thread_gimbal()
time.sleep(0.05)
myGimbal.actual_bus.shutdown()

2. Anritsu Control

In [ ]:
from a2gmeasurements import myAnritsuSpectrumAnalyzer

mySA = myAnritsuSpectrumAnalyzer()
mySA.spectrum_analyzer_connect()
rx_power =  mySA.retrieve_max_pow()
print(rx_power)
mySA.spectrum_analyzer_close()

3. Septentrio Control

In [ ]:
from a2gmeasurements import GpsSignaling
mySeptentrioGPS = GpsSignaling(DBG_LVL_2=True, DBG_LVL_1=True, DBG_LVL_0=True)

In [ ]:
mySeptentrioGPS.serial_connect()
mySeptentrioGPS.serial_instance.reset_input_buffer()


In [ ]:
#mySeptentrioGPS.sendCommandGps(cmd='sga, MultiAntenna') # by default this is the command
mySeptentrioGPS.sendCommandGps(cmd='sgd, ETRS89')
mySeptentrioGPS.sendCommandGps(cmd='spm, Rover, RTK')

In [ ]:
mySeptentrioGPS.sendCommandGps(cmd='setNTRIPSettings, NTR1, Client, opencaster.nls.fi, 2101, fuave, v5E3oKu853A, VRS-FKP')

In [ ]:

#mySeptentrioGPS.start_gps_data_retrieval(msg_type='NMEA', nmea_type='GGA', interval='sec1') # HDF instead of GGA for heading info
#mySeptentrioGPS.start_gps_data_retrieval(msg_type='NMEA', nmea_type='HDT', interval='sec1') # HDF instead of GGA for heading info

mySeptentrioGPS.start_gps_data_retrieval(msg_type='SBF', stream_number=1, interface='USB', interval='sec2')
mySeptentrioGPS.start_thread_gps()


In [ ]:
import struct
struct.calcsize('<1c3H1I1H2B3d5f1d1f4B2H1I2B4H1B')

In [ ]:
mySeptentrioGPS.stop_gps_data_retrieval(msg_type='NMEA')
mySeptentrioGPS.stop_gps_data_retrieval(msg_type='SBF')
mySeptentrioGPS.stop_thread_gps()

In [ ]:
print(len(mySeptentrioGPS.SBF_frame_buffer), len(mySeptentrioGPS.NMEA_buffer))
if len(mySeptentrioGPS.SBF_frame_buffer) > 0:
    print(mySeptentrioGPS.SBF_frame_buffer[-1])
if len(mySeptentrioGPS.NMEA_buffer) > 0:
    print(mySeptentrioGPS.NMEA_buffer[-1])

In [ ]:
mySeptentrioGPS.sendCommandGps('sdio, USB1,, -SBF-NMEA')

In [ ]:
mySeptentrioGPS.serial_instance.close()

In [ ]:
xsbf = mySeptentrioGPS.SBF_frame_buffer[-1]['X']
ybf = mySeptentrioGPS.SBF_frame_buffer[-1]['Y']
zbf = mySeptentrioGPS.SBF_frame_buffer[-1]['Z']

In [ ]:
lat=mySeptentrioGPS.NMEA_buffer[-1]['Latitude']
lon=mySeptentrioGPS.NMEA_buffer[-1]['Longitude']

In [ ]:
print(lat, lon)

In [ ]:
from a2gmeasurements import HelperA2GMeasurements


myHelper = HelperA2GMeasurements(ID=11, SERVER_ADDRESS='')


y, x = myHelper.convert_DDMMS_to_planar(input_lon=lon, 
                                                input_lat=lat, epsg_out=4936)
print(x, y)

Precision of distance when using planar coordinates ('precision' is given by google estimate of distance)

In [ ]:
import pyproj as proj
from a2gmeasurements import HelperA2GMeasurements
import numpy as np

pt_hammentie_50 = [60.186977663598164, 24.961182204780805]
pt_hakaniemi_mkt = [60.17907971164074, 24.951228689442726]

h = HelperA2GMeasurements()

lat_pln_hamm, lon_pln_hamm = h.convert_DDMMS_to_planar(24.961182204780805, 60.186977663598164)
lat_pln_hak, lon_pln_hak = h.convert_DDMMS_to_planar(24.951228689442726, 60.17907971164074)

# Compare this distance with Google MAPS: CHECKED ---> within 2m error
d_hamm_hak = np.linalg.norm(np.array([lon_pln_hamm, lat_pln_hamm]) - np.array([lon_pln_hak, lat_pln_hak]))

Example of a real GPS NMEA output

In [ ]:
real_gps_data = {'Timestamp': '110815.00', 
                 'Latitude': '6011.1011606', 
                 'Latitude Direction': 'N', 
                 'Longitude': '02448.9144475', 
                 'Longitude Direction': 'E', 
                 'GPS Quality Indicator': '2', 
                 'Number of Satellites in use': '28', 
                 'Horizontal Dilution of Precision': '0.6', 
                 'Antenna Alt above sea level (mean)': '0.9230', 
                 'Units of altitude (meters)': 'M', 
                 'Geoidal Separation': '19.5883', 
                 'Units of Geoidal Separation (meters)': 'M', 
                 'Age of Differential GPS Data (secs)': '1.8', 
                 'Differential Reference Station ID': '0123'}

Server connection

In [ ]:
from socket import socket
import time
from a2gmeasurements import HelperA2GMeasurements

host = 'localhost'
myHelper = HelperA2GMeasurements('GROUND', host)
myHelper.HelperStartA2GCom()


In [ ]:
myHelper.HelperA2GStopCom()
print(myHelper.SOCKET_BUFFER, len(myHelper.SOCKET_BUFFER))

5. Signal generator

In [ ]:
import pyvisa

rm = pyvisa.ResourceManager()
print(rm.list_resources())

inst = rm.open_resource('GPIB0::19::INSTR')
print(inst.query("*IDN?"))

inst.write('F0 28.95 GH\n')
inst.write('L0 20 DM\n')

# Turn On RF output
inst.write('RF1\n')
# Turn Off RF output
inst.write('RF0\n')

In [ ]:
import numpy as np
from timeit import default_timer as timer

def dummy_fcn(sz):
    return np.sum(np.matmul(np.random.rand(sz,sz), np.random.rand(sz,sz)), axis=(0,1))

start = timer()
tmp = dummy_fcn(10000)
print(f'FOR 10000: {timer()-  start} seconds')

start = timer()
tmp = dummy_fcn(5000)
print(f'FOR 5000: {timer()-  start} seconds')

In [ ]:
from a2gmeasurements import myAnritsuSpectrumAnalyzer

dir = 'C://Users//jvjulian//OneDrive - Teknologian Tutkimuskeskus VTT//Documents//Measurement 1//20221012133438//'
file = 'CONT20221012133504968.rsm'

anritsu = myAnritsuSpectrumAnalyzer()
anritsu.parse_xml_file(dir + file)
anritsu.XML_file

In [ ]:
from PyQt6.QtWidgets import QApplication, QLabel, QWidget
import sys

app = QApplication([])

window = QWidget()
window.setWindowTitle("PyQt App")
window.setGeometry(100, 600, 280, 280) # 1st: x coordinate, 2nd: y coordinate, 3rd: width, 4th: height
helloMsg = QLabel("<h1>Hello World</h1>", parent=window)
helloMsg.move(60, 15)
window.show()

sys.exit(app.exec())
